In [ ]:
# Load api key from .env file
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

litellm_api_key = os.getenv("LITELLM_API_KEY")
litellm_api_base = os.getenv("LITELLM_API_BASE")

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

In [ ]:

from llama_index.llms.litellm import LiteLLM

llm = LiteLLM("gpt-4o-mini", api_base=litellm_api_base, api_key=litellm_api_key)

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.helicone import Helicone

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("Some question about the data should go here")
print(response)



In [ ]:
response = query_engine.query("Who are the new owners of Drylab?")
print(response)

# Simple agent example

In [ ]:
import asyncio
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI




# Define a simple calculator tool
def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b

# Create an agent workflow with our calculator tool
agent = FunctionAgent(
    tools=[multiply],
    llm=llm,
    system_prompt="You are a helpful assistant that can multiply two numbers.",
)


response = await agent.run("What is 1234 * 4567?")
print(str(response))



In [ ]:

from llama_index.core.workflow import Context

# create context
ctx = Context(agent)

# run agent with context
response = await agent.run("My name is Logan", ctx=ctx)
print(response)
response = await agent.run("What is my name?", ctx=ctx)
print(response)